
Vamos explorar como consultar bases de dados SQL Server utilizando tanto SQL puro como Python, focando no uso do Spark (PySpark) e mencionando outras ferramentas relevantes.  O objetivo é fornecer um guia completo, desde os conceitos básicos até exemplos práticos, incluindo a criação de uma métrica calculada periodicamente.

**Parte 1: Consulta SQL Server com SQL (Diretamente)**

Para começar, é fundamental entender como consultar o SQL Server utilizando a linguagem SQL padrão. Esta é a base para qualquer interação, mesmo quando utilizamos ferramentas como Python e Spark.

**Conceitos SQL Essenciais para Consulta:**

*   **`SELECT`:**  A instrução fundamental para recuperar dados. Especifica as colunas que deseja retornar.

    ```sql
    SELECT coluna1, coluna2, ...
    FROM nome_da_tabela;
    ```

*   **`FROM`:** Indica a tabela da qual os dados serão selecionados.

*   **`WHERE`:** Filtra as linhas com base em uma ou mais condições.

    ```sql
    SELECT coluna1, coluna2
    FROM nome_da_tabela
    WHERE condição;
    ```

*   **Operadores de Comparação:**  Utilizados no `WHERE` para definir condições (`=`, `>`, `<`, `>=`, `<=`, `<>`, `!=`, `LIKE`, `IN`, `BETWEEN`, `IS NULL`, `IS NOT NULL`).

*   **Operadores Lógicos:**  Combinam condições (`AND`, `OR`, `NOT`).

*   **`JOIN`:** Combina dados de duas ou mais tabelas relacionadas.

    *   `INNER JOIN`: Retorna apenas linhas correspondentes em ambas as tabelas.
    *   `LEFT JOIN`: Retorna todas as linhas da tabela da esquerda e as correspondências da tabela da direita (se houver).
    *   `RIGHT JOIN`: Retorna todas as linhas da tabela da direita e as correspondências da tabela da esquerda (se houver).
    *   `FULL OUTER JOIN`: Retorna todas as linhas de ambas as tabelas, com correspondências sempre que possível.

    ```sql
    SELECT tabela1.coluna1, tabela2.coluna2
    FROM tabela1
    INNER JOIN tabela2 ON tabela1.coluna_chave = tabela2.coluna_chave;
    ```

*   **Funções de Agregação:** Realizam cálculos em conjuntos de dados (`COUNT`, `SUM`, `AVG`, `MIN`, `MAX`).

    ```sql
    SELECT COUNT(*), AVG(coluna_valor)
    FROM nome_da_tabela
    WHERE condição;
    ```

*   **`GROUP BY`:** Agrupa linhas com valores semelhantes em uma ou mais colunas, frequentemente usado com funções de agregação.

    ```sql
    SELECT coluna_agrupamento, COUNT(*)
    FROM nome_da_tabela
    GROUP BY coluna_agrupamento;
    ```

*   **`ORDER BY`:** Ordena os resultados por uma ou mais colunas (`ASC` para ascendente, `DESC` para descendente).

    ```sql
    SELECT coluna1, coluna2
    FROM nome_da_tabela
    ORDER BY coluna1 ASC, coluna2 DESC;
    ```

**Exemplo SQL Server Básico:**

Suponha que temos uma tabela `Vendas` no SQL Server com as colunas: `ID_Venda`, `Produto`, `Data_Venda`, `Quantidade`, `Preco_Unitario`.

**Consultas de exemplo:**

1.  **Selecionar todas as colunas e linhas:**

    ```sql
    SELECT *
    FROM Vendas;
    ```

2.  **Selecionar apenas `Produto` e `Data_Venda` para vendas após '2023-01-01':**

    ```sql
    SELECT Produto, Data_Venda
    FROM Vendas
    WHERE Data_Venda > '2023-01-01';
    ```

3.  **Calcular o total de vendas por produto:**

    ```sql
    SELECT Produto, SUM(Quantidade * Preco_Unitario) AS Total_Vendas
    FROM Vendas
    GROUP BY Produto
    ORDER BY Total_Vendas DESC;
    ```

**Parte 2: Consulta SQL Server com Python e PySpark**

PySpark é uma excelente ferramenta para trabalhar com SQL Server a partir do Python, especialmente quando lidamos com grandes volumes de dados e necessitamos de processamento distribuído e escalabilidade.

**Pré-requisitos PySpark para SQL Server:**

1.  **Instalar PySpark:** Se ainda não instalou, utilize `pip install pyspark`.
2.  **JDBC Driver para SQL Server:** PySpark utiliza JDBC para se conectar a bancos de dados relacionais como o SQL Server. Você precisará do driver JDBC para SQL Server. Pode baixar o driver da Microsoft e certificar-se de que o caminho para o arquivo JAR do driver esteja acessível ao PySpark (normalmente colocando-o na pasta `jars` do Spark ou especificando no classpath).

**Conectando ao SQL Server com PySpark:**

Utilizamos a função `spark.read.jdbc()` para ler dados do SQL Server para um DataFrame do PySpark.

In [ ]:
from pyspark.sql import SparkSession

# Inicializar SparkSession
spark = SparkSession.builder \
    .appName("SQLServerConsulta") \
    .config("spark.jars", "caminho/para/sqljdbc_auth.jar,caminho/para/mssql-jdbc-12.4.0.jre11.jar") \ # Caminho para os seus drivers JDBC
    .getOrCreate()

# Configurações de conexão JDBC
jdbcUrl = "jdbc:sqlserver://servidorSQLServer:1433;databaseName=nomeDoBancoDeDados;integratedSecurity=true;" # Para autenticação integrada do Windows
# jdbcUrl = "jdbc:sqlserver://servidorSQLServer:1433;databaseName=nomeDoBancoDeDados;" # Para autenticação SQL Server
connectionProperties = {
    "user": "seu_usuario_sqlserver", # Se usar autenticação SQL Server
    "password": "sua_senha_sqlserver", # Se usar autenticação SQL Server
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}
table_name = "nome_da_tabela_sqlserver"

# Ler dados do SQL Server para um DataFrame
df_vendas = spark.read.jdbc(url=jdbcUrl, table=table_name, properties=connectionProperties)

# Mostrar o esquema do DataFrame e as primeiras linhas
df_vendas.printSchema()
df_vendas.show()

# Encerra a SparkSession
spark.stop()

**Explicação do código:**

*   **`SparkSession.builder...config("spark.jars", ...).getOrCreate()`:** Inicializa a `SparkSession` e configura o caminho para os arquivos JAR dos drivers JDBC.  É crucial incluir os JARs corretos para que o PySpark consiga se comunicar com o SQL Server.  Você pode precisar ajustar os caminhos e os nomes dos arquivos JAR dependendo da sua versão do driver.
*   **`jdbcUrl`:** A URL de conexão JDBC para o seu SQL Server. Ajuste:
    *   `servidorSQLServer`:  Nome ou endereço IP do seu servidor SQL Server.
    *   `1433`: Porta padrão do SQL Server (altere se for diferente).
    *   `databaseName=nomeDoBancoDeDados`: Nome do banco de dados no SQL Server que deseja consultar.
    *   `integratedSecurity=true;`: Usado para autenticação integrada do Windows (o usuário que executa o Python precisa ter permissão no SQL Server). Se usar autenticação SQL Server, comente esta parte e utilize `user` e `password` em `connectionProperties`.
*   **`connectionProperties`:** Dicionário com propriedades de conexão.  Importante para fornecer credenciais se usar autenticação SQL Server (`user`, `password`) e especificar o driver JDBC (`driver`).
*   **`table_name`:** Nome da tabela no SQL Server que você deseja ler.
*   **`spark.read.jdbc(...)`:**  Função que lê dados via JDBC e cria um DataFrame.
    *   `url`: A URL de conexão JDBC.
    *   `table`: O nome da tabela ou uma subconsulta SQL (ver exemplo abaixo).
    *   `properties`: As propriedades de conexão.
*   **`df_vendas.printSchema()` e `df_vendas.show()`:** Funções para visualizar o esquema do DataFrame (tipos de dados das colunas) e as primeiras linhas de dados.
*   **`spark.stop()`:** Encerra a sessão Spark.

**Consultas SQL Diretas com `spark.sql()`:**

Você também pode executar consultas SQL diretamente no SQL Server através do PySpark utilizando `spark.sql()`, mas primeiro precisa registrar a tabela JDBC como uma tabela temporária no Spark.

In [ ]:
# ... (código de inicialização SparkSession e configurações JDBC como no exemplo anterior) ...

# Registrar tabela JDBC como uma view temporária
df_vendas.createOrReplaceTempView("vendas_temp_view")

# Executar consulta SQL diretamente usando spark.sql()
df_resultado_sql = spark.sql("""
    SELECT Produto, SUM(Quantidade * Preco_Unitario) AS Total_Vendas
    FROM vendas_temp_view
    GROUP BY Produto
    ORDER BY Total_Vendas DESC
""")

df_resultado_sql.show()

# ... (spark.stop()) ...

**Utilizando subconsultas SQL no `spark.read.jdbc()`:**

Em vez de ler uma tabela inteira, você pode usar uma subconsulta SQL diretamente no `spark.read.jdbc()` para ler apenas os dados que precisa.

In [ ]:
# ... (código de inicialização SparkSession e configurações JDBC) ...

subquery = "(SELECT Produto, Data_Venda, Quantidade, Preco_Unitario FROM Vendas WHERE Data_Venda > '2023-01-01') AS vendas_filtradas"

df_vendas_filtradas = spark.read.jdbc(url=jdbcUrl, table=subquery, properties=connectionProperties)

df_vendas_filtradas.show()

# ... (spark.stop()) ...

**Parte 3: Exemplo Prático - Cálculo Periódico de Métricas**

Vamos criar um exemplo que consulta o SQL Server periodicamente para calcular uma nova métrica: **o total de vendas do dia anterior**.

**Cenário:**

Queremos monitorar diariamente o total de vendas realizadas no dia anterior. Para isso, vamos criar um script Python que:

1.  Conecta-se ao SQL Server.
2.  Executa uma consulta para somar as vendas do dia anterior.
3.  Imprime a métrica calculada.
4.  (Em um cenário real) Poderia armazenar esta métrica em outro banco de dados, enviar um email de notificação, etc.

**Script Python com PySpark (para execução periódica):**

In [ ]:
import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum

def calcular_vendas_dia_anterior():
    """Calcula o total de vendas do dia anterior do SQL Server."""

    spark = SparkSession.builder \
        .appName("CalculoVendasDiaAnterior") \
        .config("spark.jars", "caminho/para/sqljdbc_auth.jar,caminho/para/mssql-jdbc-12.4.0.jre11.jar") \ # Ajuste o caminho dos drivers JDBC
        .getOrCreate()

    jdbcUrl = "jdbc:sqlserver://servidorSQLServer:1433;databaseName=nomeDoBancoDeDados;integratedSecurity=true;" # Ajuste a URL
    connectionProperties = {
        "user": "seu_usuario_sqlserver", # Ajuste as credenciais se necessário
        "password": "sua_senha_sqlserver",
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }

    # Calcular a data de ontem no formato 'YYYY-MM-DD' para a consulta SQL
    data_ontem = (datetime.date.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

    # Subconsulta SQL para selecionar vendas do dia anterior
    subquery_vendas_ontem = f"""
        (SELECT Quantidade, Preco_Unitario FROM Vendas WHERE CONVERT(DATE, Data_Venda) = '{data_ontem}') AS vendas_ontem
    """

    try:
        df_vendas_ontem = spark.read.jdbc(url=jdbcUrl, table=subquery_vendas_ontem, properties=connectionProperties)

        # Calcular o total de vendas do dia anterior
        total_vendas_ontem_df = df_vendas_ontem.agg(sum(df_vendas_ontem["Quantidade"] * df_vendas_ontem["Preco_Unitario"]).alias("Total_Vendas_Ontem"))
        total_vendas_ontem_row = total_vendas_ontem_df.collect()[0]
        total_vendas_ontem = total_vendas_ontem_row["Total_Vendas_Ontem"]

        print(f"Total de vendas de {data_ontem}: R$ {total_vendas_ontem:.2f}")

    except Exception as e:
        print(f"Erro ao calcular vendas do dia anterior: {e}")

    finally:
        spark.stop()

# Executar a função para calcular as vendas do dia anterior
calcular_vendas_dia_anterior()

**Passos e Explicações:**

1.  **Importar bibliotecas:** `datetime` para manipular datas, `SparkSession` e `sum` do PySpark.
2.  **Função `calcular_vendas_dia_anterior()`:** Encapsula toda a lógica.
3.  **Inicialização SparkSession e configurações JDBC:**  Similar aos exemplos anteriores.
4.  **Cálculo da data de ontem:** Usamos `datetime` para obter a data de ontem e formatá-la no formato 'YYYY-MM-DD' para usar na cláusula `WHERE` da consulta SQL.
5.  **Subconsulta SQL:** Criamos uma subconsulta SQL formatada (f-string) para selecionar apenas as linhas da tabela `Vendas` onde a `Data_Venda` corresponde ao dia anterior. `CONVERT(DATE, Data_Venda)` é usado para comparar apenas a parte da data, ignorando a hora (dependendo do tipo de dado da sua coluna `Data_Venda`).
6.  **Ler dados com `spark.read.jdbc()`:** Lemos os dados filtrados do dia anterior para um DataFrame.
7.  **Calcular total de vendas:** Usamos `df_vendas_ontem.agg(sum(...))` para calcular a soma do produto de `Quantidade` e `Preco_Unitario` (total de vendas) do DataFrame.  `alias("Total_Vendas_Ontem")` renomeia a coluna resultante.
8.  **Obter resultado:** `total_vendas_ontem_df.collect()[0]` executa a ação `collect()` para trazer o resultado (que é uma única linha e coluna neste caso) para o Driver e `.collect()[0]` pega a primeira (e única) linha. `total_vendas_ontem_row["Total_Vendas_Ontem"]` acessa o valor da coluna `Total_Vendas_Ontem` nessa linha.
9.  **Imprimir resultado:** Imprimimos a métrica calculada formatada.
10. **Tratamento de erros (`try...except`) e finalização (`finally spark.stop()`):**  Adicionamos tratamento de erros básico e garantimos que a SparkSession seja sempre encerrada, mesmo em caso de erro.
11. **Executar a função:** Chamamos `calcular_vendas_dia_anterior()` para rodar o cálculo.

**Agendamento para execução periódica:**

Para executar este script periodicamente (por exemplo, todos os dias), você pode usar ferramentas de agendamento de tarefas do sistema operacional ou ferramentas de orquestração mais robustas:

*   **Agendador de tarefas do sistema operacional (Windows Task Scheduler, cron no Linux/macOS):**  Para tarefas simples, pode agendar o script Python para rodar em um horário específico diariamente.
*   **Apache Airflow, Prefect, etc.:** Para pipelines de dados mais complexos e agendamento robusto, ferramentas de orquestração como Airflow ou Prefect são mais adequadas. Elas oferecem funcionalidades como monitoramento, tratamento de dependências entre tarefas, retentativas em caso de falha, etc.

**Parte 4: Alternativas ao PySpark para Python e SQL Server**

Embora PySpark seja excelente para processamento de grandes dados, para tarefas mais simples ou quando não há necessidade de computação distribuída, você pode considerar outras bibliotecas Python para interagir com SQL Server:

*   **`pyodbc`:** Uma das bibliotecas mais populares para conectar Python a bancos de dados ODBC, incluindo SQL Server.  É mais leve que PySpark e geralmente mais fácil de configurar para consultas simples.

In [ ]:
import pyodbc

    def calcular_vendas_dia_anterior_pyodbc():
        """Calcula vendas do dia anterior usando pyodbc."""
        try:
            conexao = pyodbc.connect(
                'DRIVER={SQL Server};'
                'SERVER=servidorSQLServer;'
                'DATABASE=nomeDoBancoDeDados;'
                'UID=seu_usuario_sqlserver;'
                'PWD=sua_senha_sqlserver;' # Ou usar autenticação Windows: 'Trusted_Connection=yes;'
            )
            cursor = conexao.cursor()

            data_ontem = (datetime.date.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
            query_sql = f"""
                SELECT SUM(Quantidade * Preco_Unitario) AS Total_Vendas_Ontem
                FROM Vendas
                WHERE CONVERT(DATE, Data_Venda) = '{data_ontem}'
            """
            cursor.execute(query_sql)
            resultado = cursor.fetchone()
            total_vendas_ontem = resultado[0] if resultado[0] else 0

            print(f"Total de vendas de {data_ontem}: R$ {total_vendas_ontem:.2f}")

        except Exception as e:
            print(f"Erro ao calcular vendas do dia anterior com pyodbc: {e}")

        finally:
            if conexao:
                conexao.close()

    calcular_vendas_dia_anterior_pyodbc()

*   **`SQLAlchemy`:** Uma biblioteca ORM (Object-Relational Mapper) poderosa que oferece uma forma mais abstrata de interagir com bancos de dados. Permite definir modelos de dados em Python e realizar operações de banco de dados de forma mais orientada a objetos, além de suportar SQL "raw". É mais complexo que `pyodbc` para consultas simples, mas oferece mais flexibilidade para aplicações maiores e complexas.

**Quando usar PySpark vs. Alternativas?**

*   **PySpark:** Ideal para:
    *   Grandes volumes de dados (Big Data) que não cabem na memória de uma única máquina.
    *   Processamento distribuído e escalável.
    *   Tarefas de análise de dados complexas, transformações, agregações em larga escala.
    *   Integração com outras funcionalidades do Spark (MLlib para Machine Learning, Spark Streaming para processamento em tempo real).

*   **`pyodbc`, `SQLAlchemy`:**  Mais adequados para:
    *   Volumes de dados menores ou que cabem na memória da máquina.
    *   Tarefas mais simples de consulta, inserção, atualização de dados.
    *   Aplicações que não exigem processamento distribuído.
    *   Desenvolvimento mais rápido e configuração mais simples para casos de uso menores.

**Conclusão:**

Este material detalhou como consultar o SQL Server utilizando SQL padrão e Python com PySpark.  Demonstramos exemplos práticos, incluindo o cálculo periódico de uma métrica, e mencionamos alternativas como `pyodbc` para cenários onde o Spark pode ser excessivo. A escolha da ferramenta dependerá do volume de dados, complexidade da tarefa e requisitos de escalabilidade do seu projeto.